# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

First update to the latest version of AzureML SDK

In [ ]:
!pip install --upgrade azureml-sdk[automl]

In [10]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

In [11]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config()

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [12]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-project-bank-churn'

experiment=Experiment(ws, experiment_name)

In [13]:
# Quick check to see what datasets are already registerd
for a in ws.datasets.keys():
    print(a)

In [14]:
# get external data https://raw.githubusercontent.com/fuzzballb/nd00333-capstone/master/starter_file/cleaned_data.csv

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bank-churn"
description_text = "Bank churn DataSet for Udacity Course"

if key in ws.datasets.keys():
        print("using existing dataset") 
        found = True
        dataset = ws.datasets[key] 

if not found:
        print("retrieving dataset") 
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/fuzzballb/nd00333-capstone/master/starter_file/cleaned_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

retrieving dataset


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.454300,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.501400,0.250900,0.247700
std,96.653299,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.500023,0.433553,0.431698
min,350.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,0.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,1.000000,0.000000,0.000000
75%,718.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,1.000000,0.000000
max,850.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                    "max_concurrent_iterations":4,
                    "iterations": 10,
                    "primary_metric" : 'accuracy'                     
}
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = compute_target,
    task="classification",
    training_data = dataset,
    label_column_name= "Exited",
    n_cross_validations=5,
    **automl_settings)   

In [16]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_58efcc0b-8669-4f80-a5c5-cff132d1afe7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detec

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [47]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-bank-churn,AutoML_58efcc0b-8669-4f80-a5c5-cff132d1afe7,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [48]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [58]:
best_automl_run,fitted_model = remote_run.get_output()
print(best_automl_run)
print(fitted_model)


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: udacity-project-bank-churn,
Id: AutoML_58efcc0b-8669-4f80-a5c5-cff132d1afe7_8,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_leaf_nodes=None,
                                                                                                    max_samples=None,
              

TypeError: predict() missing 1 required positional argument: 'X'

In [60]:
#TODO: Save the best model

# import os

# if not os.path.exists("outputs"):
#     os.makedirs("outputs")
import joblib
joblib.dump(value=fitted_model, filename='automl_test_experiment_best_model2.pkl')

Registering model Regression_model_new


In [5]:
from azureml.core.model import Model
model = Model.register(model_path="automl_test_experiment_best_model2.pkl",
                        model_name="Regression_model_new",
                        tags={'area': "churn", 'type': "regression"},
                        description="Model to predict churn",
                        workspace=ws)

Registering model Regression_model_new


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment.from_conda_specification(name='automlv2',file_path='conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4, auth_enabled = True, enable_app_insights = True)

In [21]:
service_name = 'bankchurnpredictionautoml8'
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)
print(service.state)

service.wait_for_deployment(show_output = True)
print(service.scoring_uri)

Transitioning
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://57ea7fd1-a3c6-4513-b540-ea076b3a1780.southcentralus.azurecontainer.io/score


In [22]:
service.get_logs()

'2021-02-21T10:35:22,228292096+00:00 - rsyslog/run \n2021-02-21T10:35:22,228476804+00:00 - nginx/run \n2021-02-21T10:35:22,227457662+00:00 - gunicorn/run \n2021-02-21T10:35:22,228947224+00:00 - iot-server/run \n/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
import json

#CreditScore	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	[Exited]	Geography_France	Geography_Germany	Geography_Spain

#502	1	42	8	159660.8	3	1	0	113931.57	[1] 1	0	0
#543	0	22	8	0	        2	0	0	127587.22	[0]	1	0	0

data = {"data":
        [
          {
            "CreditScore": 	502,
            "Gender": 1,	
            "Age": 42,	
            "Tenure": 8,	
            "Balance": 159660.8,	
            "NumOfProducts":	3,
            "HasCrCard":	1,
            "IsActiveMember": 0,	
            "EstimatedSalary": 113931.57,	
            "Geography_France": 1,	
            "Geography_Germany": 0,	
            "Geography_Spain": 0
          },
          {
            "CreditScore": 	543,
            "Gender": 0,	
            "Age": 22,	
            "Tenure": 8,	
            "Balance": 0,	
            "NumOfProducts":	2,
            "HasCrCard":	0,
            "IsActiveMember": 0,	
            "EstimatedSalary": 127587.22,	
            "Geography_France": 1,	
            "Geography_Germany": 0,	
            "Geography_Spain": 0
          }
      ]
    }

inputJson = json.dumps(data)
serviceOutput = service.run(inputJson)

print(serviceOutput)

{"result": [1, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()